In [39]:
import numpy as np
from tensorflow.keras import Model as Model_
from tensorflow.keras.layers import Input, PReLU, LeakyReLU, MaxPooling2D, Dropout, concatenate, UpSampling2D, ReLU, Conv2D, Flatten, Reshape, Conv1D, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Conv2DTranspose, Dense, BatchNormalization
from tensorflow.keras.datasets import mnist, cifar10
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import pandas as pd


# torch.manual_seed(0) # Set for testing purposes, please do not change!

print(tf.__version__)

2.1.0


In [131]:
direction_csv_file = pd.read_csv("Directions 1.54138969.csv", header=None)

In [132]:
direction_csv_file

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,473.6836,444.9424,500.9961,448.0299,479.8393,530.7856,506.2184,622.5688,493.6608,621.9954,...,515.4715,456.4298,515.4715,456.4298,499.2511,448.2281,515.0607,479.1209,515.0607,479.1209
1,473.6876,444.9526,500.9847,448.0307,479.8167,530.7855,506.2182,622.5726,493.6592,621.9958,...,514.9991,456.2006,514.9991,456.2006,498.6736,447.8242,514.2874,478.6892,514.2874,478.6892
2,473.6931,444.9663,500.9775,448.0403,479.7976,530.7957,506.2265,622.5868,493.6538,621.9990,...,514.7042,456.0592,514.7042,456.0592,498.3504,447.4883,513.7126,478.3876,513.7126,478.3876
3,473.7039,444.9854,500.9819,448.0659,479.7655,530.8015,506.2534,622.5867,493.6528,622.0079,...,514.5616,455.9726,514.5616,455.9726,498.1750,447.2817,513.3622,478.2127,513.3622,478.2127
4,473.7261,445.0067,501.0016,448.0924,479.7466,530.8076,506.2579,622.5981,493.6506,622.0058,...,514.6009,455.9780,514.6009,455.9780,498.1767,447.2362,513.2590,478.2043,513.2590,478.2043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,508.8683,435.9992,531.6415,432.5145,531.7281,525.6750,536.5560,619.7991,507.8583,621.3291,...,552.5535,450.4184,552.5535,450.4184,535.6805,447.9043,557.0486,475.3463,557.0486,475.3463
1379,508.8894,435.9756,531.7077,432.5069,531.7758,525.6630,536.5682,619.7924,507.8712,621.3207,...,551.9902,450.3370,551.9902,450.3370,535.2592,447.4315,556.0175,475.3270,556.0175,475.3270
1380,508.9090,435.9860,531.6987,432.5204,531.7960,525.6733,536.5767,619.8024,507.8655,621.3173,...,551.3666,450.2951,551.3666,450.2951,534.9061,446.8693,554.7869,475.3963,554.7869,475.3963
1381,508.9323,435.9888,531.6935,432.5264,531.8196,525.6773,536.5802,619.8096,507.8681,621.3137,...,550.6077,450.2511,550.6077,450.2511,534.4236,446.4702,553.4522,475.5750,553.4522,475.5750


In [136]:
direction_numpy = np.asarray(direction_csv_file)

In [151]:
downsampled_data = downsample_dataset(direction_numpy)

In [152]:
downsampled_data.shape

(691, 64)

In [150]:
#Function to downsample the dataset to half of its size
def downsample_dataset(dataset) :
    return np.delete(dataset, np.s_[::2], axis=0)

In [40]:
import os
import re

class Dataset_Preprocessing:
    def __init__(self, dir_path, include_dimension = 2, sample_size = 50, total_classes = 17):
        
        #Dataset Directory path
        self.dir_path = dir_path
        
        #Which Dimension file to include, possible values: 2 and 3
        self.include_dimension = include_dimension
        
        #Total frames in one Sample
        self.sample_size = sample_size
        
        #Activity classes to include
        self.classes = ['SittingDown', 'Walking', 'Directions', 'Discussion', 'Sitting', 'Phoning', 'Eating', 'Posing', 'Greeting', 'Smoking']
        
        #Total activity classes
        self.total_classes = len(self.classes)
        
        #Subject Folders names in the Dataset
        self.internal_folders = ['S1', 'S5','S6','S7','S8','S9','S11']
    
    def read_dataset(self):
        try:
            #Contains all the different activity vectors
            activity_vector = {}
            
            #Contains the overall dataset
            sampled_data = None
            
            #Based on dimensions, which folder to use for extracting the dataset files
            data_folder = 'Poses_D2_Positions' if self.include_dimension == 2 else 'Poses_D3_Positions'
            
            #Checking if the dataset path is valid
            if not os.path.exists(self.dir_path):
                print('The Data Directory Does not Exist!')
                return None

            #Iterating over all the subject folders
            for fld in self.internal_folders:
                #Iterating for each file in the specified folder
                for file in os.listdir(os.path.join(self.dir_path, fld, data_folder)):
                    #Extracting the activity from the filename
                    activity = self.__extract_activity(file)
                    
                    if activity not in self.classes:
                        continue
                    
                    #Reading the CSV file using Pandas
                    data = pd.read_csv(os.path.join(self.dir_path, fld, data_folder, file), header=None)

                    #Formulating the activity vector using one hot encoding
                    if activity not in activity_vector:
                        total_keys = len(activity_vector.keys())
                        activity_vector[activity] = np.zeros(self.total_classes)
                        activity_vector[activity][total_keys] = 1
                    vector = activity_vector[activity]
                    
                    #Sampling the dataset
                    grouped_sample = self.__group_samples(data, self.sample_size, vector)
                    sampled_data = grouped_sample if sampled_data is None else np.append(sampled_data, grouped_sample, axis=0)
                    
            return sampled_data
        except Exception as e:
            print(e)
    
    def __extract_activity(self, filename):
        try:
            #Extracting the filename and excluding the extension
            name = os.path.splitext(filename)[0]
            
            #Substituting the empty string with characters other than english alphabets
            activity = re.sub('[^A-Za-z]+' , '' , name)
            return activity
        except Exception as e:
            print(e)
    
    def __group_samples(self, dataset, sample_size, activity):
        try:
            #Checking if the dataset is a Pandas Dataframe
            if not isinstance(dataset, pd.DataFrame):
                print('Expecting Pandas Dataframe, but got {}'.format(type(dataset)))
                return None
            
            #Appending activity class to each row in the dataset
            dataset = pd.concat([dataset, pd.DataFrame(np.tile(activity, (dataset.shape[0],1)))], axis=1)
            
            #Reshaping the dataset into sample batches
            total_samples = dataset.shape[0]//sample_size
            total_features = dataset.shape[1]
            grouped_rows = dataset.to_numpy()[:total_samples*self.sample_size].reshape((-1,self.sample_size, total_features))
            
            return grouped_rows
        except Exception as e:
            print(e)

In [41]:
sampled_data = Dataset_Preprocessing('H3.6csv').read_dataset()

In [42]:
sampled_data.shape

(30688, 50, 74)

In [43]:
print('Total Samples: {}'.format(sampled_data.shape[0]))
print('Total Frames: {}'.format(sampled_data.shape[1]))
print('Total Features: {}'.format(sampled_data.shape[2]))

Total Samples: 30688
Total Frames: 50
Total Features: 74


In [6]:
validation_set = sampled_data[np.s_[::10], :, :]

In [7]:
validation_set.shape

(3069, 50, 74)

In [80]:
#Split the data into training, validation and test 
def split_data(data_array, validation_size=0.1, test_size=0.1) :
    assert validation_size+test_size < 0.5 , f"total size of validation and testing set should not exceed half of the dataset"
    assert validation_size > 0, f"validation size should be greater than zero"
    assert test_size > 0, f"test size should be greater than zero"
    
    validation_step = int(1/validation_size)
    test_step = int(1/test_size)
    
    
    mask = np.ones(data_array.shape, dtype=bool)
    validation_data = data_array[np.s_[::validation_step], :, :]
    mask[np.s_[1::validation_step],:,:] = False
    test_data = data_array[np.s_[1::test_step], :, :]
    mask[np.s_[::test_step],:,:] = False
    training_data = data_array[mask]
    training_data_size = data_array.shape[0] - (validation_data.shape[0]+test_data.shape[0])
    training_data = training_data.reshape((training_data_size, data_array.shape[1], data_array.shape[2]))
    return training_data, validation_data, test_data

In [130]:
def split_to_features_labels(dataset, input_sequance_size=10) :
    assert input_sequance_size < dataset.shape[1], f"input sequance should be smaller than the total sample size"
    features = dataset[:, np.s_[0:input_sequance_size], :]
    labels = dataset[:,np.s_[input_sequance_size:], :]
    
    return features, labels

In [81]:
training_data, validation_data, test_data = split_data(sampled_data)

(30688, 50, 74)
(30688, 50, 74)


In [113]:
print("Training data shape is ", training_data.shape)
print("Validation data shape is ", validation_data.shape)
print("Test data shape is ", test_data.shape)

Training data shape is  (24550, 50, 74)
Validation data shape is  (3069, 50, 74)
Test data shape is  (3069, 50, 74)


In [131]:
training_dataX, training_dataY = split_to_features_labels(training_data, input_sequance_size=10)

In [133]:
training_dataX.shape

(24550, 10, 74)

In [121]:
class GloGen_Bidirectional_RNN_encoder(Model_):
    def __init__(self, num_recurrent_neurons=200):
        super(GloGen_Bidirectional_RNN_encoder, self).__init__()

        #Return Sequances=True to assure return of output corresponding to each timestep
        self.recurrent_layer = LSTM(num_recurrent_neurons, return_sequences=True)

def call(self, input_x):
    output = self.recurrent_layer(input_x)
    return output

In [114]:
#Not sure of the output shape from the glonet
class GloGen_Bidirectional_RNN_decoder(Model_):
  def __init__(self, num_recurrent_neurons=64):
    super(GloGen_Bidirectional_RNN_decoder, self).__init__()
    #Return Sequances=True to assure return of output corresponding to each timestep
    self.recurrent_layer = LSTM(num_recurrent_neurons, return_sequences=True)

  def call(self, encoder_output):
    glogen_output = self.recurrent_layer(encoder_output)
    #output = tf.one_hot(tf.argmax(output, axis = 1), depth=3)
    return glogen_output

In [115]:
class LocGen_Bidirectional_RNN_encoder(Model_) :
  def __init__(self, num_recurrent_neurons=200) :
    super(LocGen_Bidirectional_RNN_encoder, self).__init__()
    #Return Sequances=True to assure return of output corresponding to each timestep
    self.recurrent_layer = LSTM(num_recurrent_neurons, return_sequences=True)


  def call(self, interpolated_output) :
    predictions = self.recurrent_layer(interpolated_output)
    return predictions

In [116]:
class LocGen_Bidirectional_RNN_decoder(Model_) :
  def __init__(self, num_recurrent_neurons=64) :
    super(LocGen_Bidirectional_RNN_decoder, self).__init__()
    #Return Sequances=True to assure return of output corresponding to each timestep
    self.recurrent_layer = LSTM(num_recurrent_neurons, return_sequences=True)

  def call(self, encoder_output) :
    final_predictions = self.recurrent_layer(encoder_output)
    return final_predictions

In [122]:
glogen_bidirectional_RNN_Encoder = GloGen_Bidirectional_RNN_encoder()

In [123]:
#Reshaping direction data array
training_data.shape

(24550, 50, 74)

In [69]:
#Should have diminsions 100x60x64 as batch_siZe x timesteps x features
direction_data_array.shape

(10, 10, 64)

In [70]:
#simulating applying the glogen encoder on an input batch with batch size "10"
output_encoder = glogen_bidirectional_RNN_Encoder(direction_data_array)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [71]:
#shape of 10x200 predictions for each sample inside the batch
output_encoder.shape

TensorShape([10, 10, 200])

In [72]:
#Simulating the output shape for the decoder
glogen_decoder = GloGen_Bidirectional_RNN_decoder()

In [73]:
glogen_sparse_output_predictions = glogen_decoder(output_encoder)

In [74]:
#Prediction for 32 joints position for the next 10 timesteps for each sample inside the batch
glogen_sparse_output_predictions.shape

TensorShape([10, 10, 64])

In [75]:
#returns a list of the interpolated frames between x0 and x1
def interpolateFrames(glogen_sparse_output_predictions, num_of_frames=5) :
  batch_size = glogen_sparse_output_predictions.shape[0]
  timesteps = glogen_sparse_output_predictions.shape[1]
  features = glogen_sparse_output_predictions.shape[2]
  interpolated_frames = np.zeros((batch_size, timesteps, num_of_frames, features))
  for batch in range(glogen_sparse_output_predictions.shape[0]) :
    for t in range(glogen_sparse_output_predictions.shape[1]-1) :
      for j in range(num_of_frames) :
        X_i0 = glogen_sparse_output_predictions[batch, t]
        X_i1 = glogen_sparse_output_predictions[batch, t+1]
        
        alpha_j = j/num_of_frames
        current_frame = alpha_j*X_i0 + (1-alpha_j)*X_i1
        interpolated_frames[batch, t, j] = current_frame

  return interpolated_frames 

In [76]:
interpolated_frames = interpolateFrames(glogen_sparse_output_predictions)

In [77]:
#Shape of interpolated frames should be 10(batch size) x 10(timesteps)
#x 5(num of interpolated frames between these timesteps) x 64(num_features)
interpolated_frames.shape

(10, 10, 5, 64)

In [78]:
#timesteps and interpolated frames diminsion can be flattened to 10(batch size) x 50(dense frames) x 64(features)
interpolated_frames = np.reshape(interpolated_frames, (10, 50, 64))

In [79]:
#Simulating the locgen phase with batches as well
locgen_encoder = LocGen_Bidirectional_RNN_encoder()
locgen_encoder_output = locgen_encoder(interpolated_frames)
print("loc gen encoder output is ", locgen_encoder_output.shape)

locgen_decoder = LocGen_Bidirectional_RNN_decoder()
final_predictions = locgen_decoder(locgen_encoder_output)

print("final predictions shape is ", final_predictions.shape)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

loc gen encoder output is  (10, 50, 200)
final predictions shape is  (10, 50, 64)


In [127]:
class Optimizer:
    def __init__(self, model, mb = 8, lr = 0.00001, loss = tf.keras.losses.MeanSquaredError, opt=tf.keras.optimizers.Adam, 
               lambda_1=0.5, lambda_2=0.5):
        self.model     = model
        self.loss      = loss() #tf.keras.losses.MeanSquaredError()
        self.optimizer = opt(learning_rate = lr)
        self.mb        = mb

        self.lambda_1 = lambda_1
        self.lambda_2 = lambda_2

        self.train_loss     = tf.keras.metrics.Mean(name='train_loss')
        self.train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

        self.test_loss     = tf.keras.metrics.Mean(name='test_loss')
        self.test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

  
    @tf.function
    def train_step(self, x , y):
        with tf.GradientTape() as tape:
            predictions = self.model(x)
            loss = self.loss(y, predictions)

        #print(self.model.trainable_variables)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        self.train_loss(loss)
        self.train_accuracy(y, predictions)
        return loss

    @tf.function
    def test_step(self, x , y):
        predictions = self.model(x)
        loss = self.loss(y, predictions)
        self.test_loss(loss)
        self.test_accuracy(y, predictions)

    def train(self):
        for mbX, mbY in self.train_ds:
            self.train_step(mbX, mbY)

    def test(self):
        for mbX, mbY in self.test_ds:
            self.test_step(mbX, mbY)    

    def run(self, dataX, dataY, testX, testY, epochs, verbose=2):
        historyTR = []
        historyTS = []
        template = '{} {}, {}: {}, {}: {}'
        self.train_ds = tf.data.Dataset.from_tensor_slices((dataX, dataY)).shuffle(16000).batch(self.mb)
        self.test_ds  = tf.data.Dataset.from_tensor_slices((testX,testY)).batch(self.mb)
        for i in range(epochs):

            self.train ()
            #   print(lossTR)
            self.test  ()
            if verbose > 0:
                print(template.format("epoch: ", i+1,
                              #" TRAIN LOSS: ", self.train_loss.result(),
                              #" TEST LOSS: " , self.test_loss.result()))
                               " TRAIN ACC: " , self.train_accuracy.result()*100,
                               " TEST ACC: "  , self.test_accuracy.result()*100) )

            temp = '{}'
            historyTR.append(float(temp.format(self.train_loss.result())))
            historyTS.append(float(temp.format(self.test_loss.result() )))

            self.train_loss.reset_states()
            self.train_accuracy.reset_states()
            self.test_loss.reset_states()
            self.test_accuracy.reset_states()
        return historyTR, historyTS

    #Defining the loss function utilities
    def loss_joint(self, predicted_sequance_batch, target_sequance_batch) :
        diff_norm_2 = np.sum(np.square(np.subtract(final_predictions, real_sequance)), axis=2)
        return np.sum(diff_norm_2, axis=1) 

    def loss_motion_flow(self, predicted_sequance_batch, target_sequance_batch) :
        predictions_tomporal_diffs = np.diff(predicted_sequance_batch, axis=1)
        real_tomporal_diffs = np.diff(target_sequance_batch, axis=1)
        prediction_motion_flow_diff_norm_2 = np.sum(np.square(np.subtract(final_predictions, real_sequance)), axis=2)
        return np.sum(prediction_motion_flow_diff_norm_2, axis=1)

    def total_loss(self, predicted_sequance_batch, target_sequance_batch) :
        joints_loss = self.loss_joint(predicted_sequance_batch, target_sequance_batch)
        motion_flow_loss = self.motion_flow_loss(predicted_sequance_batch, target_sequance_batch)
        return joints_loss*self.lambda_1 + motion_flow_loss*self.lambda_2
